In [1]:
import pandas as pd
import numpy as np
from selenium import webdriver
import time

In [2]:
races = pd.read_csv('races.csv')

In [3]:
races.head()

,season,round,circuit_id,lat,long,country,date,url
0,1950,1,silverstone,52.0786,-1.01694,UK,1950-05-13,http://en.wikipedia.org/wiki/1950_British_Gran...
1,1950,2,monaco,43.7347,7.42056,Monaco,1950-05-21,http://en.wikipedia.org/wiki/1950_Monaco_Grand...
2,1950,3,indianapolis,39.7950,-86.23470,USA,1950-05-30,http://en.wikipedia.org/wiki/1950_Indianapolis...
3,1950,4,bremgarten,46.9589,7.40194,Switzerland,1950-06-04,http://en.wikipedia.org/wiki/1950_Swiss_Grand_...
4,1950,5,spa,50.4372,5.97139,Belgium,1950-06-18,http://en.wikipedia.org/wiki/1950_Belgian_Gran...


In [4]:
weather = races.iloc[:,[0,1,2]]

In [6]:
info = []

for link in races.url:
    try:
        df = pd.read_html(link)[0]
        if 'Weather' in list(df.iloc[:,0]):
            n = list(df.iloc[:,0]).index('Weather')
            info.append(df.iloc[n,1])
        else:
            df = pd.read_html(link)[1]
            if 'Weather' in list(df.iloc[:,0]):
                n = list(df.iloc[:,0]).index('Weather')
                info.append(df.iloc[n,1])
            else:
                df = pd.read_html(link)[2]
                if 'Weather' in list(df.iloc[:,0]):
                    n = list(df.iloc[:,0]).index('Weather')
                    info.append(df.iloc[n,1])
                else:
                    df = pd.read_html(link)[3]
                    if 'Weather' in list(df.iloc[:,0]):
                        n = list(df.iloc[:,0]).index('Weather')
                        info.append(df.iloc[n,1])
                    else:
                        driver = webdriver.Chrome()
                        driver.get(link)

                        # click language button
                        button = driver.find_element_by_link_text('Italiano')
                        button.click()
                        
                        clima = driver.find_element_by_xpath('//*[@id="mw-content-text"]/div/table[1]/tbody/tr[9]/td').text
                        info.append(clima) 
                                
    except:
        info.append('not found')

In [7]:
len(info)

1079

In [8]:
weather['weather'] = info

<ipython-input-8-0345b68f31a0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  weather['weather'] = info


In [9]:
weather.head()

,season,round,circuit_id,weather
0,1950,1,silverstone,"Sunny, mild, dry."
1,1950,2,monaco,not found
2,1950,3,indianapolis,Rainy
3,1950,4,bremgarten,"Warm, dry and sunny"
4,1950,5,spa,"Warm, dry and sunny"


In [10]:
weather.tail()

,season,round,circuit_id,weather
1074,2022,18,suzuka,Rain
1075,2022,19,americas,Partly cloudy
1076,2022,20,rodriguez,Partly cloudy
1077,2022,21,interlagos,Cloudy
1078,2022,22,yas_marina,Clear


In [11]:
weather_dict = {'weather_warm': ['soleggiato', 'clear', 'warm', 'hot', 'sunny', 'fine', 'mild', 'sereno'],
               'weather_cold': ['cold', 'fresh', 'chilly', 'cool'],
               'weather_dry': ['dry', 'asciutto'],
               'weather_wet': ['showers', 'wet', 'rain', 'pioggia', 'damp', 'thunderstorms', 'rainy'],
               'weather_cloudy': ['overcast', 'nuvoloso', 'clouds', 'cloudy', 'grey', 'coperto']}

In [12]:
weather_df = pd.DataFrame(columns = weather_dict.keys())

In [13]:
for col in weather_df:
    weather_df[col] = weather['weather'].map(lambda x: 1 if any(i in weather_dict[col] for i in x.lower().split()) else 0)

In [14]:
weather_df.head()

,weather_warm,weather_cold,weather_dry,weather_wet,weather_cloudy
0,0,0,0,0,0
1,0,0,0,0,0
2,0,0,0,1,0
3,1,0,1,0,0
4,1,0,1,0,0


In [16]:
weather_info = pd.concat([weather, weather_df], axis = 1)

weather_info.head()

,season,round,circuit_id,weather,weather_warm,weather_cold,weather_dry,weather_wet,weather_cloudy
0,1950,1,silverstone,"Sunny, mild, dry.",0,0,0,0,0
1,1950,2,monaco,not found,0,0,0,0,0
2,1950,3,indianapolis,Rainy,0,0,0,1,0
3,1950,4,bremgarten,"Warm, dry and sunny",1,0,1,0,0
4,1950,5,spa,"Warm, dry and sunny",1,0,1,0,0


In [18]:
weather_info.tail()

,season,round,circuit_id,weather,weather_warm,weather_cold,weather_dry,weather_wet,weather_cloudy
1074,2022,18,suzuka,Rain,0,0,0,1,0
1075,2022,19,americas,Partly cloudy,0,0,0,0,1
1076,2022,20,rodriguez,Partly cloudy,0,0,0,0,1
1077,2022,21,interlagos,Cloudy,0,0,0,0,1
1078,2022,22,yas_marina,Clear,1,0,0,0,0


In [19]:
weather_info.to_csv('weather.csv', index= False)